In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [9]:
# Discriminator model
class Discriminator(tf.keras.Model):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=img_shape),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1)
        ])

    def call(self, x):
        return self.model(x)


In [10]:
# Generator model
class Generator(tf.keras.Model):
    def __init__(self, z_shape):
        super(Generator, self).__init__()
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(7*7*512, input_dim=z_shape),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Reshape((7, 7, 512)),
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(256, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(128, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Conv2D(1, kernel_size=3, padding='same'),
            tf.keras.layers.Activation('tanh')
        ])

    def call(self, z):
        return self.model(z)

In [11]:
# DCGAN class
class DCGAN:
    def __init__(self, img_shape, epochs=50000, lr_gen=0.0001, lr_disc=0.0001, z_shape=100, batch_size=64, epochs_for_sample=5000):
        self.img_shape = img_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.z_shape = z_shape
        self.epochs_for_sample = epochs_for_sample
        self.generator = Generator(self.z_shape)
        self.discriminator = Discriminator(self.img_shape)

        # Optimizers
        self.gen_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_gen)
        self.disc_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_disc)

    def train(self, X_train):
        for epoch in range(self.epochs):
            idx = np.random.randint(0, len(X_train), self.batch_size)
            batch_X = X_train[idx]
            batch_Z = np.random.uniform(-1, 1, (self.batch_size, self.z_shape))

            # Train Discriminator
            with tf.GradientTape() as disc_tape:
                disc_logits_real = self.discriminator(batch_X)
                disc_logits_fake = self.discriminator(self.generator(batch_Z))
                disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.ones_like(disc_logits_real), logits=disc_logits_real)) + \
                             tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.zeros_like(disc_logits_fake), logits=disc_logits_fake))

            grads_disc = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
            self.disc_optimizer.apply_gradients(zip(grads_disc, self.discriminator.trainable_variables))

            # Train Generator
            with tf.GradientTape() as gen_tape:
                gen_imgs = self.generator(batch_Z)
                disc_logits_fake = self.discriminator(gen_imgs)
                gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                    labels=tf.ones_like(disc_logits_fake), logits=disc_logits_fake))

            grads_gen = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
            self.gen_optimizer.apply_gradients(zip(grads_gen, self.generator.trainable_variables))

            # Generate samples every 5000 epochs
            if epoch % self.epochs_for_sample == 0:
                self.generate_sample(epoch)

            if epoch % 100 == 0:
                print(f"Epoch: {epoch}, Discriminator Loss: {disc_loss.numpy()}, Generator Loss: {gen_loss.numpy()}")

    def generate_sample(self, epoch):
        z = np.random.uniform(-1, 1, (self.batch_size, self.z_shape))
        imgs = self.generator(z)
        imgs = (imgs.numpy() * 0.5 + 0.5)  # Scale back to [0, 1]

        for i in range(self.batch_size):
            img = imgs[i, :, :, 0]  # Select the individual image
            label = np.random.randint(0, 10)  # Replace with the actual label you want to associate
            plt.imshow(img, cmap="gray")
            plt.axis('off')
            plt.title(f'Label: {label}')  # Show label on the image
            
            # Save individual images with label in the filename
            plt.savefig(f"samples/{epoch}_img_{i}_label_{label}.png", bbox_inches='tight')
            plt.close()

In [12]:
# Load and preprocess data
def load_data():
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')

    X_train = train_df.iloc[:, 1:].values.astype(np.float32)  # Exclude the label column
    y_train = train_df.iloc[:, 0].values  # Labels

    X_test = test_df.iloc[:, 1:].values.astype(np.float32)  # Exclude the label column
    y_test = test_df.iloc[:, 0].values  # Labels

    # Preprocess data: scale to [-1, 1] and reshape
    X_train = (X_train / 255.0) * 2 - 1  # Scale to [-1, 1]
    X_train = X_train.reshape(-1, 28, 28, 1)  # Reshape to (samples, 28, 28, 1)

    return X_train, y_train, X_test, y_test


In [13]:
# Main execution
if __name__ == '__main__':
    img_shape = (28, 28, 1)  # Shape of the images
    X_train, y_train, X_test, y_test = load_data()

    # Create samples directory
    if not os.path.exists("samples"):
        os.makedirs("samples")

    dcgan = DCGAN(img_shape)
    dcgan.train(X_train)


C:\Users\msadi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\msadi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
C:\Users\msadi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch: 0, Discriminator Loss: 1.3481866121292114, Generator Loss: 0.6917556524276733
Epoch: 100, Discriminator Loss: 0.0016148430295288563, Generator Loss: 6.676881790161133
Epoch: 200, Discriminator Loss: 0.0005105931777507067, Generator Loss: 8.006937980651855
Epoch: 300, Discriminator Loss: 0.4793214201927185, Generator Loss: 1.011662483215332
Epoch: 400, Discriminator Loss: 0.8543341159820557, Generator Loss: 2.0203163623809814
Epoch: 500, Discriminator Loss: 0.008884875103831291, Generator Loss: 5.362359046936035
Epoch: 600, Discriminator Loss: 0.003193259472027421, Generator Loss: 6.215540409088135
Epoch: 700, Discriminator Loss: 0.0025000236928462982, Generator Loss: 6.625896453857422
Epoch: 800, Discriminator Loss: 0.002011875854805112, Generator Loss: 7.139761924743652
Epoch: 900, Discriminator Loss: 0.0012219077907502651, Generator Loss: 7.6170806884765625
Epoch: 1000, Discriminator Loss: 1.1001336574554443, Generator Loss: 1.2697343826293945
Epoch: 1100, Discriminator Loss: 

KeyboardInterrupt: 